In [1]:
from IPython.display import HTML
from dataset import TUBerlinDataset
from prepare_data import convert_and_quantize_svg

labels = ["backpack"]
dataset = TUBerlinDataset(labels, download=True)

originals = ""
quantized = ""

for svg_content in dataset[0:3]:
    originals += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original, length: {len(svg_content)}</b><br>{svg_content}</div>'
    output = convert_and_quantize_svg(svg_content, bins=256)
    quantized += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Quantized, length: {len(output)}</b><br>{output}</div>'

display(HTML(originals))
display(HTML(quantized))

Loading TUBerlin files: 100%|██████████| 1/1 [00:00<00:00,  2.09it/s]


In [2]:
from IPython.display import HTML
from dataset import QuickDrawDataset
from prepare_data import convert_and_quantize_svg

labels = ["bulldozer"]
dataset = QuickDrawDataset(labels, download=True)

originals = ""
quantized = ""

for svg_content in dataset[0:3]:
    originals += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original, length: {len(svg_content)}</b><br>{svg_content}</div>'
    output = convert_and_quantize_svg(svg_content, bins=128)
    quantized += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Quantized, length: {len(output)}</b><br>{output}</div>'

display(HTML(originals))
display(HTML(quantized))

Loading QuickDraw files: 100%|██████████| 1/1 [00:04<00:00,  4.96s/it]


## Point-based representation

In [2]:
from IPython.display import HTML
from utils import svg_to_tensor, tensor_to_svg, svg_to_tensor_quantized

original_svg = """<svg viewBox="0 0 256 256"><g stroke-width="0.8">
<path d="M 26 22 L 32 50 L 33 84" stroke="black" fill="none"/>
<path d="M 24 24 L 56 16 L 114 7 L 113 75 L 111 82 L 105 86" stroke="black" fill="none"/>
<path d="M 8 101 L 26 96 L 130 84" stroke="black" fill="none"/>
<path d="M 7 103 L 6 153 L 11 156 L 24 157 L 59 150 L 127 147" stroke="black" fill="none"/>
<path d="M 138 86 L 132 142 L 134 148" stroke="black" fill="none"/>
<path d="M 133 85 L 133 78 L 157 18 L 173 0 L 189 24 L 203 57 L 207 91 L 196 106 L 195 126 L 203 134 L 214 138 L 229 139 L 237 136 L 247 128 L 240 105 L 226 89 L 209 87 L 204 92" stroke="black" fill="none"/>
<path d="M 202 140 L 209 147 L 211 146 L 215 127 L 221 141 L 225 140 L 232 130 L 238 137 L 247 124 L 255 121" stroke="black" fill="none"/>
<path d="M 201 141 L 201 141" stroke="black" fill="none"/>
<path d="M 200 128 L 204 137" stroke="black" fill="none"/>
<path d="M 2 163 L 38 150 L 155 137 L 163 139 L 166 149 L 166 156 L 158 166 L 148 173 L 121 180 L 41 191 L 23 191 L 7 185 L 1 176 L 1 160 L 14 152" stroke="black" fill="none"/>
</g></svg>"""
tensor = svg_to_tensor(original_svg)
reconstructed_svg = tensor_to_svg(tensor)

print("svg -> tensor -> svg")
svgs_inline = f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original</b><br>{original_svg}</div>'
svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Reconstructed</b><br>{reconstructed_svg}</div>'
display(HTML(svgs_inline))

# Note: quantization is faster when converting directly from svg to quantized tensor
print("svg -> quantized tensor -> svg")
quantization_levels = [16, 32, 64, 128, 256]
svgs_inline = ""
for bins in quantization_levels:
    quantized_tensor = svg_to_tensor_quantized(original_svg, bins=bins)
    reconstructed_svg = tensor_to_svg(quantized_tensor, size=bins)
    svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;">{reconstructed_svg}</div>'
display(HTML(svgs_inline))

# print(tensor.shape)
# print(tensor)
# print(tensor.nbytes)

# If quantized to 128 bins, the tensor can be stored as int8

svg -> tensor -> svg


svg -> quantized tensor -> svg


In [ ]:
# Using a point based representation there a half as many points needed to be represented
# dx, dy is good because it is relative but it leads to a larger tokenizer vocab size
from utils import AbsolutePenPositionTokenizer

tokenizer = AbsolutePenPositionTokenizer(bins=64)
print(len(tokenizer.vocab))

encoded = tokenizer.encode(original_svg)
print(f"Encoded length: {len(encoded)}")
print(f"Encoded tokens: {encoded}")

decoded_svg = tokenizer.decode(encoded)

print(decoded_svg)

print("svg -> encoder / decoder -> svg")
svgs_inline = f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original</b><br>{original_svg}</div>'
svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Reconstructed</b><br>{decoded_svg}</div>'
display(HTML(svgs_inline))


4100
Encoded length: 80
Encoded tokens: [4098, 4096, 389, 524, 533, 4096, 390, 900, 1794, 1810, 1748, 1685, 4096, 153, 408, 2069, 4096, 153, 102, 230, 423, 997, 2020, 4096, 2197, 2083, 2148, 4096, 2133, 2131, 2500, 2752, 3014, 3214, 3286, 3098, 3103, 3233, 3426, 3618, 3745, 3936, 3802, 3606, 3285, 3223, 4096, 3234, 3300, 3364, 3423, 3491, 3554, 3680, 3810, 3935, 4062, 4096, 3171, 3171, 4096, 3168, 3234, 4096, 40, 613, 2466, 2594, 2661, 2662, 2537, 2347, 1964, 687, 431, 174, 43, 39, 229, 4099]
<svg viewBox="0 0 64 64"><g stroke-width="0.8">
<path d="M 6 5 L 8 12 L 8 21" stroke="black" fill="none"/>
<path d="M 6 6 L 14 4 L 28 2 L 28 18 L 27 20 L 26 21" stroke="black" fill="none"/>
<path d="M 2 25 L 6 24 L 32 21" stroke="black" fill="none"/>
<path d="M 2 25 L 1 38 L 3 38 L 6 39 L 15 37 L 31 36" stroke="black" fill="none"/>
<path d="M 34 21 L 32 35 L 33 36" stroke="black" fill="none"/>
<path d="M 33 21 L 33 19 L 39 4 L 43 0 L 47 6 L 50 14 L 51 22 L 48 26 L 48 31 L 50 33 L 53 34 L 56 34 L 5

## Stroke-based representation

In [4]:
from IPython.display import HTML
from utils import (
    svg_strokes_to_tensor,
    svg_strokes_to_tensor_quantized,
    tensor_to_svg_strokes,
)
import torch

original_svg = """<svg viewBox="0 0 256 256"><g stroke-width="0.8">
<path d="M 26 22 L 32 50 L 33 84" stroke="black" fill="none"/>
<path d="M 24 24 L 56 16 L 114 7 L 113 75 L 111 82 L 105 86" stroke="black" fill="none"/>
<path d="M 8 101 L 26 96 L 130 84" stroke="black" fill="none"/>
<path d="M 7 103 L 6 153 L 11 156 L 24 157 L 59 150 L 127 147" stroke="black" fill="none"/>
<path d="M 138 86 L 132 142 L 134 148" stroke="black" fill="none"/>
<path d="M 133 85 L 133 78 L 157 18 L 173 0 L 189 24 L 203 57 L 207 91 L 196 106 L 195 126 L 203 134 L 214 138 L 229 139 L 237 136 L 247 128 L 240 105 L 226 89 L 209 87 L 204 92" stroke="black" fill="none"/>
<path d="M 202 140 L 209 147 L 211 146 L 215 127 L 221 141 L 225 140 L 232 130 L 238 137 L 247 124 L 255 121" stroke="black" fill="none"/>
<path d="M 201 141 L 201 141" stroke="black" fill="none"/>
<path d="M 200 128 L 204 137" stroke="black" fill="none"/>
<path d="M 2 163 L 38 150 L 155 137 L 163 139 L 166 149 L 166 156 L 158 166 L 148 173 L 121 180 L 41 191 L 23 191 L 7 185 L 1 176 L 1 160 L 14 152" stroke="black" fill="none"/>
</g></svg>"""
tensor = svg_strokes_to_tensor(original_svg)
tensor = tensor.to(torch.int16)
reconstructed_svg = tensor_to_svg_strokes(tensor)

print("svg -> tensor -> svg")
svgs_inline = f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Original</b><br>{original_svg}</div>'
svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;"><b>Reconstructed</b><br>{reconstructed_svg}</div>'
display(HTML(svgs_inline))

# Note: quantization is faster when converting directly from svg to quantized tensor
print("svg -> quantized tensor -> svg")
quantization_levels = [16, 32, 64, 128, 256]
svgs_inline = ""
for bins in quantization_levels:
    quantized_tensor = svg_strokes_to_tensor_quantized(original_svg, bins=bins)
    reconstructed_svg = tensor_to_svg_strokes(quantized_tensor, size=bins)
    svgs_inline += f'<div style="display:inline-block; width: 150px; background-color: white; margin-right:10px;">{reconstructed_svg}</div>'
display(HTML(svgs_inline))

# print(tensor.shape)
# print(tensor)
# print(tensor.nbytes)

svg -> tensor -> svg


svg -> quantized tensor -> svg


In [12]:
from IPython.display import HTML
from dataset import SketchyDataset

sketchy_dataset = SketchyDataset(labels=["cat"], download=True)
sample = sketchy_dataset[0]

svgs_inline = f'<div style="display:inline-block; width: 150px; margin-right:10px;"><b>Sample</b><br>{sample}</div>'
display(HTML(svgs_inline))

Loading Sketchy files: 100%|██████████| 1/1 [00:00<00:00,  7.11it/s]
